In [1]:
import os
import pandas as pd
import tensorflow
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import TextVectorization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/toxity/train.csv")
test = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/toxity/test.csv")
test_lable= pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/toxity/test_labels.csv")
train.head()

In [ ]:
x = train['comment_text']
y = train[train.columns[2:]].values
MAX_FEATURES = 200000  # number of words
vectorizer = TextVectorization(max_tokens = MAX_FEATURES,
                               output_mode = 'int',
                               output_sequence_length = 1800
                               )

In [ ]:
vectorizer.adapt(x.values)
vectorized_text = vectorizer(x.values)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [ ]:
dataset_train = dataset.take(int(len(dataset)*.7))
dataset_val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.3))

In [ ]:
train_generator = dataset.as_numpy_iterator()

# 2. Training

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [ ]:
model = Sequential([Embedding(MAX_FEATURES+1, 32),
                    Bidirectional(LSTM(32,activation='tanh')),
                    Dense(128,activation='relu'),
                    Dense(256,activation='relu'),
                    Dense(128,activation='relu'),
                    Dense(6,activation='sigmoid')])
model.compile(loss=tf.metrics.binary_crossentropy,optimizer='Adam')
model.summary()

In [ ]:
history = model.fit(dataset_train, epochs=10, validation_data=dataset_val)

In [ ]:
model.save('savedmodel.h5')

# 3. Make predictions

In [ ]:
test = vectorizer(test['comment_text'])
yhat = np.around(model.predict(test))

# 4. Evaluating

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [ ]:
pre = Precision()
rec = Recall()
acc = CategoricalAccuracy()

In [ ]:
y = test_lable[test_lable.columns[1:]].values.flatten()
yhat = yhat.flatten()
pre.update_state(y, yhat)
rec.update_state(y, yhat)
acc.update_state(y, yhat)
print('Precision: %f\nRecall: %f\nAccuracy: %f\n' % (pre.result(), rec.result(), acc.result()))

In [16]:
pre = Precision()
rec = Recall()
acc = CategoricalAccuracy()

In [38]:
y = test_lable[test_lable.columns[1:]].values.flatten()
yhat = yhat.flatten()
pre.update_state(y, yhat)
rec.update_state(y, yhat)
acc.update_state(y, yhat)
print('Precision: %f\nRecall: %f\nAccuracy: %f\n' % (pre.result(), rec.result(), acc.result()))

Precision: 0.868370
Recall: 0.117774
Accuracy: 0.986672

